# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,279.51,88,75,2.24,RU,1697574926
1,1,at taj,24.2049,23.2857,294.25,30,0,4.73,LY,1697574926
2,2,hilo,19.7297,-155.0900,301.62,73,75,4.12,US,1697574850
3,3,waitangi,-43.9535,-176.5597,284.72,76,8,1.79,NZ,1697574927
4,4,vila franca do campo,37.7167,-25.4333,294.20,99,80,5.26,PT,1697574927


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size='Humidity')

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# my ideal weather conditions: cold and cloudy but not too damp
# so i'll find cities where max temp is below mid-60s(Fahrenheit), where humidity is low and cloudiness is high

# max temperature is in units of kelvin
# 290 kelvin is approximately 62 fahrenheit, we want rows with max temp below 290
ideal_weather_df = city_data_df[city_data_df['Max Temp'] < 290]

# humidity less than 50%
ideal_weather_df = ideal_weather_df[ideal_weather_df['Humidity'] < 50]

# cloudiness greater than 60% 
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness'] > 60]

# per the hint on the challenge page on canvas, I chose these requirements to get 10 or fewer (in this case 9) cities

# Drop any rows with null values
ideal_weather_df.dropna(how='any', inplace=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,kamloops,50.6665,-120.3192,289.70,49,75,7.20,CA,1697574714
142,142,zavodskoy,43.4597,132.2922,282.57,43,96,2.06,RU,1697574985
208,208,seymour,38.9592,-85.8903,287.75,41,67,2.61,US,1697575013
244,244,arvayheer,46.2639,102.7750,273.96,28,99,2.71,MN,1697575027
371,371,shalqar,47.8333,59.6000,284.16,43,100,3.13,KZ,1697575081
421,421,hastings,50.8552,0.5729,286.44,31,100,10.04,GB,1697575101
482,482,la ronge,55.1001,-105.2842,289.27,47,75,2.06,CA,1697575125
492,492,choibalsan,48.0667,114.5000,277.79,40,100,5.57,MN,1697575129
508,508,hotan,37.0997,79.9269,285.15,28,98,3.27,CN,1697575136


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
76,76,kamloops,50.6665,-120.3192,289.70,49,75,7.20,CA,1697574714,
142,142,zavodskoy,43.4597,132.2922,282.57,43,96,2.06,RU,1697574985,
208,208,seymour,38.9592,-85.8903,287.75,41,67,2.61,US,1697575013,
244,244,arvayheer,46.2639,102.7750,273.96,28,99,2.71,MN,1697575027,
371,371,shalqar,47.8333,59.6000,284.16,43,100,3.13,KZ,1697575081,
421,421,hastings,50.8552,0.5729,286.44,31,100,10.04,GB,1697575101,
482,482,la ronge,55.1001,-105.2842,289.27,47,75,2.06,CA,1697575125,
492,492,choibalsan,48.0667,114.5000,277.79,40,100,5.57,MN,1697575129,
508,508,hotan,37.0997,79.9269,285.15,28,98,3.27,CN,1697575136,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# 10,000 meter radius
radius = 10000
params = {
    # categories: hotel, "accommodation hotel" from documentation
    # accommodation has two m's, not one. oops
    "categories": "accommodation.hotel",
    # limit: how many results do we want, just one
    "limit": 1,
    # filter: this needs to be set up using lat and long so cant be done here
    # bias: this needs to be set up using lat and long so cant be done here
    # api key
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{row["Lng"]},{row["Lat"]},{radius}'
    params["bias"] = f'proximity:{row["Lng"]},{row["Lat"]}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kamloops - nearest hotel: No hotel found
zavodskoy - nearest hotel: No hotel found
seymour - nearest hotel: Quality Inn & Suites
arvayheer - nearest hotel: Time Hotel
shalqar - nearest hotel: Бекнур
hastings - nearest hotel: White Rock Hotel
la ronge - nearest hotel: La Ronge Hotel and Suites
choibalsan - nearest hotel: Зүүн Ордон Зочид Буудал
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
76,76,kamloops,50.6665,-120.3192,289.70,49,75,7.20,CA,1697574714,No hotel found
142,142,zavodskoy,43.4597,132.2922,282.57,43,96,2.06,RU,1697574985,No hotel found
208,208,seymour,38.9592,-85.8903,287.75,41,67,2.61,US,1697575013,Quality Inn & Suites
244,244,arvayheer,46.2639,102.7750,273.96,28,99,2.71,MN,1697575027,Time Hotel
371,371,shalqar,47.8333,59.6000,284.16,43,100,3.13,KZ,1697575081,Бекнур
421,421,hastings,50.8552,0.5729,286.44,31,100,10.04,GB,1697575101,White Rock Hotel
482,482,la ronge,55.1001,-105.2842,289.27,47,75,2.06,CA,1697575125,La Ronge Hotel and Suites
492,492,choibalsan,48.0667,114.5000,277.79,40,100,5.57,MN,1697575129,Зүүн Ордон Зочид Буудал
508,508,hotan,37.0997,79.9269,285.15,28,98,3.27,CN,1697575136,Hua Yi International Hotel (accepts foreigners)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', global_extent=True, 
                              hover_cols=["Hotel Name", "Country"])

# Display the map
map2

# I'm not happy with this block of errors I get, but the map still displays
# to do a little sleuthing, all of those are the same error, and they reference my environment?
# it's duplicated 9 times: once for each row in my hotels_df dataframe?
# anyways I've tried about 5 different things to resolve this, I'm leaving it as is. sorry :(

C:\Users\stars\anaconda3\envs\PythonData_2023_08\lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
C:\Users\stars\anaconda3\envs\PythonData_2023_08\lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
C:\Users\stars\anaconda3\envs\PythonData_2023_08\lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
C:\Users\stars\anaconda3\envs\PythonData_2023_08\lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)